# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
from pandas.tseries.offsets import *
import holidays
import pandas_datareader.data as web

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
import yfinance as yf
yf.pdr_override()

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
#.SA para as BR
ticker = yf.Ticker(f'{symbol}')
stockinfo = ticker.info
stockinfo

data = web.get_data_yahoo(f'{symbol}')
data = data[['Close']]
    
#print(data.Close.iloc[len(data)-1])


[*********************100%***********************]  1 of 1 completed



## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data.Close.iloc[len(data)-1]

148.24000549316406

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 50))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO
AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI
CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTSH,CTVA,CTXS,CVS,CVX,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED
EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLS,FLT,FMC,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN
GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,IP,IPG,IPGP,IQV,

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
#final_dataframe = pd.DataFrame(columns = my_columns)
#for symbol in stocks['Ticker']:
#    print(symbol)
#    data = web.get_data_yahoo(f'{symbol}')
#    pricetoday = data.Close.iloc[len(data)-1]
#    if len(data) >= 300:
#        priceoneyear = data.Close.iloc[len(data)-300]
#    else :
#        priceoneyear = data.Close.iloc[1]
#    oneyeareturn = priceoneyear - pricetoday
#    final_dataframe = final_dataframe.append(
#    pd.Series(
#         [
#         symbol,
#         pricetoday,
#         oneyeareturn,
#         'N/A'
#         ],
#         index = my_columns),
#    ignore_index = True,    
#    )
#    
#final_dataframe

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
#final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
#final_dataframe = final_dataframe[:50]
#final_dataframe.reset_index(inplace = True)
#final_dataframe

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
portfolio_size = 1000000
# portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [9]:
#position_size = val/len(final_dataframe.index)
#for i in range(0, len(final_dataframe.index)):
#    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i,'Price'])
#
#final_dataframe

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'Two-Month Price Return',
    'Two-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol in stocks['Ticker']:
    data = web.get_data_yahoo(f'{symbol}')
    pricetoday = data.Close.iloc[len(data)-1]
    print(symbol)
    if len(data) >= 300:
        priceoneyear = data.Close.iloc[len(data)-300]
    else :
        priceoneyear = data.Close.iloc[1]
    if len(data) >= 22*6:
        pricesixmonth = data.Close.iloc[len(data)-22*6]
    else :
        pricesixmonth = data.Close.iloc[1]
    if len(data) >= 22*3:
        pricethreemonth = data.Close.iloc[len(data)-22*3]
    else :
        pricethreemonth = data.Close.iloc[1]
    if len(data) >= 22*2:
        pricetwomonth = data.Close.iloc[len(data)-22*2]
    else :
        pricetwomonth = data.Close.iloc[1]
    twomonthreturn = pricetoday - pricetwomonth
    threemonthreturn = pricetoday - pricethreemonth
    sixmonthreturn = pricetoday - pricesixmonth
    oneyeareturn = pricetoday - priceoneyear
    hqm_dataframe = hqm_dataframe.append(
         pd.Series(
         [
             symbol,
             pricetoday,
             'N/A',
             oneyeareturn,
             'N/A',
             sixmonthreturn,
             'N/A',
             threemonthreturn,
             'N/A',
             twomonthreturn,
             'N/A',
             'N/A'
         ],
         index = hqm_columns),
     ignore_index = True
     )

hqm_dataframe        

[*********************100%***********************]  1 of 1 completed
A
[*********************100%***********************]  1 of 1 completed
AAL
[*********************100%***********************]  1 of 1 completed
AAP
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%***********************]  1 of 1 completed
ABC
[*********************100%***********************]  1 of 1 completed
ABMD
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ACN
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
ADI
[*********************100%***********************]  1 of 1 completed
ADM
[*********************100%***********************]  1 of 1 completed
ADP
[*********************100%***********************

[*********************100%***********************]  1 of 1 completed
CSCO
[*********************100%***********************]  1 of 1 completed
CSX
[*********************100%***********************]  1 of 1 completed
CTAS
[*********************100%***********************]  1 of 1 completed
CTSH
[*********************100%***********************]  1 of 1 completed
CTVA
[*********************100%***********************]  1 of 1 completed
CTXS
[*********************100%***********************]  1 of 1 completed
CVS
[*********************100%***********************]  1 of 1 completed
CVX
[*********************100%***********************]  1 of 1 completed
D
[*********************100%***********************]  1 of 1 completed
DAL
[*********************100%***********************]  1 of 1 completed
DD
[*********************100%***********************]  1 of 1 completed
DE
[*********************100%***********************]  1 of 1 completed
DFS
[*********************100%***********************]

[*********************100%***********************]  1 of 1 completed
HUM
[*********************100%***********************]  1 of 1 completed
HWM
[*********************100%***********************]  1 of 1 completed
IBM
[*********************100%***********************]  1 of 1 completed
ICE
[*********************100%***********************]  1 of 1 completed
IDXX
[*********************100%***********************]  1 of 1 completed
IEX
[*********************100%***********************]  1 of 1 completed
IFF
[*********************100%***********************]  1 of 1 completed
ILMN
[*********************100%***********************]  1 of 1 completed
INCY
[*********************100%***********************]  1 of 1 completed
INFO
[*********************100%***********************]  1 of 1 completed
INTC
[*********************100%***********************]  1 of 1 completed
INTU
[*********************100%***********************]  1 of 1 completed
IP
[*********************100%********************

[*********************100%***********************]  1 of 1 completed
NWS
[*********************100%***********************]  1 of 1 completed
NWSA
[*********************100%***********************]  1 of 1 completed
O
[*********************100%***********************]  1 of 1 completed
ODFL
[*********************100%***********************]  1 of 1 completed
OKE
[*********************100%***********************]  1 of 1 completed
OMC
[*********************100%***********************]  1 of 1 completed
ORCL
[*********************100%***********************]  1 of 1 completed
ORLY
[*********************100%***********************]  1 of 1 completed
OTIS
[*********************100%***********************]  1 of 1 completed
OXY
[*********************100%***********************]  1 of 1 completed
PAYC
[*********************100%***********************]  1 of 1 completed
PAYX
[*********************100%***********************]  1 of 1 completed
PBCT
[*********************100%*******************

[*********************100%***********************]  1 of 1 completed
UPS
[*********************100%***********************]  1 of 1 completed
URI
[*********************100%***********************]  1 of 1 completed
USB
[*********************100%***********************]  1 of 1 completed
V
[*********************100%***********************]  1 of 1 completed
VFC
[*********************100%***********************]  1 of 1 completed
VIAC
[*********************100%***********************]  1 of 1 completed
VLO
[*********************100%***********************]  1 of 1 completed
VMC
[*********************100%***********************]  1 of 1 completed
VNO
[*********************100%***********************]  1 of 1 completed
VRSK
[*********************100%***********************]  1 of 1 completed
VRSN
[*********************100%***********************]  1 of 1 completed
VRTX
[*********************100%***********************]  1 of 1 completed
VTR
[*********************100%***********************

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,A,156.000000,N/A,58.400002,N/A,18.660004,N/A,3.330002,N/A,-20.729996,N/A,N/A
1,AAL,19.135000,N/A,5.695001,N/A,-2.905001,N/A,-2.035000,N/A,-0.375000,N/A,N/A
2,AAP,225.919998,N/A,68.779999,N/A,28.779999,N/A,12.529999,N/A,24.779999,N/A,N/A
3,AAPL,148.264999,N/A,22.407501,N/A,13.544998,N/A,2.625000,N/A,-4.854996,N/A,N/A
4,ABBV,114.313004,N/A,19.803001,N/A,2.923004,N/A,-4.556999,N/A,-6.256996,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
485,YUM,124.660004,N/A,28.100006,N/A,8.980003,N/A,-5.649994,N/A,-6.029999,N/A,N/A
486,ZBH,141.669998,N/A,4.860001,N/A,-34.050003,N/A,-21.070007,N/A,-8.110001,N/A,N/A
487,ZBRA,532.270020,N/A,256.880005,N/A,33.770020,N/A,-15.829956,N/A,-52.510010,N/A,N/A
488,ZION,63.490002,N/A,30.450001,N/A,8.880001,N/A,10.650002,N/A,6.290001,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'Two-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])        
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,A,156.000000,N/A,58.400002,74.6939,18.660004,74.898,3.330002,56.7347,-20.729996,6.53061,N/A
1,AAL,19.135000,N/A,5.695001,19.1837,-2.905001,29.3878,-2.035000,35.102,-0.375000,48.1633,N/A
2,AAP,225.919998,N/A,68.779999,79.3878,28.779999,81.4286,12.529999,79.7959,24.779999,92.449,N/A
3,AAPL,148.264999,N/A,22.407501,46.3265,13.544998,68.9796,2.625000,54.6939,-4.854996,26.1224,N/A
4,ABBV,114.313004,N/A,19.803001,43.0612,2.923004,48.1633,-4.556999,26.3265,-6.256996,21.8367,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
485,YUM,124.660004,N/A,28.100006,54.6939,8.980003,61.8367,-5.649994,21.8367,-6.029999,22.8571,N/A
486,ZBH,141.669998,N/A,4.860001,17.9592,-34.050003,4.08163,-21.070007,6.32653,-8.110001,18.1633,N/A
487,ZBRA,532.270020,N/A,256.880005,97.1429,33.770020,84.0816,-15.829956,8.97959,-52.510010,2.2449,N/A
488,ZION,63.490002,N/A,30.450001,56.9388,8.880001,61.2245,10.650002,76.5306,6.290001,74.6939,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_precentiles = []
    for time_period in time_periods:
        momentum_precentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_precentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,A,156.000000,N/A,58.400002,74.6939,18.660004,74.898,3.330002,56.7347,-20.729996,6.53061,53.2143
1,AAL,19.135000,N/A,5.695001,19.1837,-2.905001,29.3878,-2.035000,35.102,-0.375000,48.1633,32.9592
2,AAP,225.919998,N/A,68.779999,79.3878,28.779999,81.4286,12.529999,79.7959,24.779999,92.449,83.2653
3,AAPL,148.264999,N/A,22.407501,46.3265,13.544998,68.9796,2.625000,54.6939,-4.854996,26.1224,49.0306
4,ABBV,114.313004,N/A,19.803001,43.0612,2.923004,48.1633,-4.556999,26.3265,-6.256996,21.8367,34.8469
...,...,...,...,...,...,...,...,...,...,...,...,...
485,YUM,124.660004,N/A,28.100006,54.6939,8.980003,61.8367,-5.649994,21.8367,-6.029999,22.8571,40.3061
486,ZBH,141.669998,N/A,4.860001,17.9592,-34.050003,4.08163,-21.070007,6.32653,-8.110001,18.1633,11.6327
487,ZBRA,532.270020,N/A,256.880005,97.1429,33.770020,84.0816,-15.829956,8.97959,-52.510010,2.2449,48.1122
488,ZION,63.490002,N/A,30.450001,56.9388,8.880001,61.2245,10.650002,76.5306,6.290001,74.6939,67.3469


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,GOOGL,2953.639893,N/A,1368.489868,99.7959,643.709961,100,238.089844,100,61.829834,98.7755,99.6429
1,AZO,1786.560059,N/A,576.630005,99.3878,342.680054,99.5918,160.650024,99.1837,230.850098,100,99.5408
2,GOOG,2957.699951,N/A,1369.500000,100,630.959961,99.7959,226.889893,99.7959,48.310059,97.3469,99.2347
3,SIVB,718.739990,N/A,469.379990,98.5714,145.809998,98.1633,152.029968,98.9796,159.039978,99.7959,98.8776
4,INTU,622.909973,N/A,289.789978,97.7551,204.619965,99.1837,97.399963,98.1633,57.339966,97.9592,98.2653
...,...,...,...,...,...,...,...,...,...,...,...,...
485,AMGN,206.490005,N/A,-29.080002,1.42857,-49.029999,1.63265,-35.970001,4.08163,-17.639999,8.77551,3.97959
486,GPN,143.317505,N/A,-29.002502,1.63265,-75.042496,1.02041,-49.952499,2.44898,-17.982498,8.57143,3.41837
487,LMT,330.079987,N/A,-64.960022,0.612245,-41.900024,2.65306,-42.520020,3.06122,-31.970001,4.08163,2.60204
488,TFX,353.839996,N/A,-17.010010,2.65306,-86.170013,0.408163,-42.160004,3.26531,-38.739990,3.26531,2.39796


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portifolio = 1000000
#portifolio_input()

In [15]:
position_size = float(portifolio)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
     hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Two-Month Price Return,Two-Month Return Percentile,HQM Score
0,GOOGL,2953.639893,0,1368.489868,99.7959,643.709961,100,238.089844,100,61.829834,98.7755,99.6429
1,AZO,1786.560059,1,576.630005,99.3878,342.680054,99.5918,160.650024,99.1837,230.850098,100,99.5408
2,GOOG,2957.699951,0,1369.500000,100,630.959961,99.7959,226.889893,99.7959,48.310059,97.3469,99.2347
3,SIVB,718.739990,2,469.379990,98.5714,145.809998,98.1633,152.029968,98.9796,159.039978,99.7959,98.8776
4,INTU,622.909973,3,289.789978,97.7551,204.619965,99.1837,97.399963,98.1633,57.339966,97.9592,98.2653
...,...,...,...,...,...,...,...,...,...,...,...,...
485,AMGN,206.490005,9,-29.080002,1.42857,-49.029999,1.63265,-35.970001,4.08163,-17.639999,8.77551,3.97959
486,GPN,143.317505,14,-29.002502,1.63265,-75.042496,1.02041,-49.952499,2.44898,-17.982498,8.57143,3.41837
487,LMT,330.079987,6,-64.960022,0.612245,-41.900024,2.65306,-42.520020,3.06122,-31.970001,4.08163,2.60204
488,TFX,353.839996,5,-17.010010,2.65306,-86.170013,0.408163,-42.160004,3.26531,-38.739990,3.26531,2.39796


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momemtum_strategy_SP500.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer,sheet_name = "Momemtum Strategy Br IBOV", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats ={
       'A': ['Ticker', string_template],
       'B': ['Price', dollar_template],
       'C': ['Number of Shares to Buy', integer_template],
       'D': ['One-Year Price Return', percent_template],
       'E': ['One-Year Return Percentile', percent_template],
       'F': ['Six-Month Price Return', percent_template],
       'G': ['Six-Month Return Percentile', percent_template],
       'H': ['Three-Month Price Return', percent_template],
       'I': ['Three-Month Return Percentile', percent_template],
       'J': ['One-Month Price Return', percent_template],
       'K': ['One-Month Return Percentile', percent_template],
       'L': ['HQM Score', integer_template]        
}

for column in column_formats.keys():
    writer.sheets['Momemtum Strategy Br IBOV'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momemtum Strategy Br IBOV'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()